In [1]:
import sys
sys.path.insert(0, '/home/yw440/MXGPY/mxgpy')
sys.path.insert(0, '/home/yw440/MyRGPs/')

import os
os.environ['MXNET_ENGINE_TYPE'] = 'NaiveEngine'

import scipy.io
from pylab import *
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import GPy
import numpy as np
np.random.seed(101)

from mxgpy.rgp.rgp import RGP 
from mxgpy.kernel.rbf import RBF
import mxnet as mx
ctx = mx.gpu()
mx.Context.default_ctx = ctx

 /anaconda/envs/py35/lib/python3.5/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning:OpenSSL.rand is deprecated - you should use os.urandom instead
 /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/context.py:144: DeprecationWarning:Context.default_ctx has been deprecated. Please use Context.current_context() instead. Please use test_utils.set_default_context to set a default context


In [2]:
from mxgpy.util import positive_transform_reverse_numpy
import helper

 /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/context.py:144: DeprecationWarning:Context.default_ctx has been deprecated. Please use Context.current_context() instead. Please use test_utils.set_default_context to set a default context


## RGP - consider joint correlations, train on walk and running
- Each joint is dependednt on its parent joint, except for root which doesn't have a parent joint. 
- Each parent joint is treated as a control signal to model the correlation between the joints.
- For root, the control signal is chosen as a constant to represent the walking speed. 

In [ ]:
train_motions = ['01', '02', '03', '04', '17', '18', '19', '20']
test_motions = ['05', '06', '07', '08', '21', '22', '07', '08']
subject = '35'
y_bones = ['root', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rfemur', 'rtibia', 'rfoot', 'rtoes', 
             'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 
             'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb',
            'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rfingers', 'rthumb']
# list of control signal in order
u_bones = ['None', 'root', 'lfemur', 'ltibia', 'lfoot', 'root', 'rfemur', 'rtibia', 'rfoot', 
          'root', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck',
          'thorax', 'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lwrist',
          'thorax', 'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rwrist' ]

m1 = GPy.util.datasets.cmu_mocap(subject, train_motions, test_motions, sample_every=3, data_set='cmu_mocap')
y_train = helper.extract_independent_bones(m1['Y'], m1['lbls']) # list of lists of arrays
y_test = helper.extract_independent_bones(m1['Ytest'], m1['lblstest'])


y_train_dict = {'None':None}
y_train_mean_list, y_train_std_list = [],[]
for i, item in enumerate(y_train):
    out, mean, std = helper.normalise_data(item)
    y_train_dict[y_bones[i]] = out
    y_train_mean_list.append(mean)
    y_train_std_list.append(std)
       
y_test_dict = {}
y_test_mean_list, y_test_std_list = [], []
for i, item in enumerate(y_test):
    out, mean, std = helper.normalise_data(item)
    y_test_dict[y_bones[i]] = out
    y_test_mean_list.append(mean)
    y_test_std_list.append(std)
    

 /home/yw440/MyRGPs/helper.py:42: RuntimeWarning:invalid value encountered in true_divide


In [ ]:
for j, bone in enumerate(y_bones):  
    print(str(j)+'-'+bone)
    y_train = y_train_dict[bone]
    if bone is 'root':
        u_train = [np.ones((np.shape(y_train[0])[0],1)),
                   np.ones((np.shape(y_train[1])[0],1)),
                   np.ones((np.shape(y_train[2])[0],1)),
                   np.ones((np.shape(y_train[3])[0],1)),
                   np.ones((np.shape(y_train[4])[0],1)),
                   np.ones((np.shape(y_train[5])[0],1)),
                   np.ones((np.shape(y_train[6])[0],1)),
                   np.ones((np.shape(y_train[7])[0],1))] # number of training sequences
    else:
        u_train = y_train_dict[u_bones[j]] 
        
    YD = np.shape(y_train[0])[1] # dimension of the data, 62 dimension
    UD = np.shape(u_train[0])[1] # dimension of the control signal
     
    ## Initialise model
    win_out = 20 # only consider the feature from previous time step
    win_in = 20
    m_control = RGP(wins=[win_out], with_control=True, X_dims=[YD], U_dim=UD, U_win=win_in, num_inducing=100,
                    kernels=[RBF(win_out*YD+win_in*UD, ARD=True)], ctx=ctx, dtype=np.float64)
    
    ## Initialize parameters
    m_control.collect_params()['rbf_variance'].set_data(positive_transform_reverse_numpy(np.array([3.]), np.float64))
    m_control.collect_params()['rbf_lengthscale'].set_data(positive_transform_reverse_numpy(np.ones((win_out*YD+win_in*UD,))*5,np.float64))
    m_control.collect_params()['layer_0_noise_var'].set_data(positive_transform_reverse_numpy(np.array([0.0001]),np.float64))

    ## Train model
    m_control.optimize(Y=y_train, U=u_train, max_iters=500, step_rate=0.2)
    m_control.collect_params()['layer_0_noise_var'].grad_req = 'null' # fix the noise variance to a single value
    m_control.optimize(Y=y_train, U=u_train, max_iters=500, step_rate=0.1)
    m_control.collect_params()['layer_0_noise_var'].grad_req = 'write' # write new noise variance to a single value
    m_control.optimize(Y=y_train, U=u_train, max_iters=1500, step_rate=0.05)

    m_control.save_params('./models/fully_observed/'+str(bone))

0-root
55490303.69469773
41542405.66192631
30911930.47451069
22439747.925493196
15802509.234502235
10847052.766323756
7340122.43293773
4979043.711926236
3447142.9038047264
2468282.4783844585
1839458.1905138392
1426406.2178878547
1145135.3416188709
945074.5705959324
796551.956977998
682136.2741347206
591533.136317625
518372.2748648904
458259.34246335144
408068.8212341758
365628.6354483454
329400.33590364375
298246.5098284054
271293.9292407973
247852.80548310882
227366.8293540367
209381.3526355032
193522.37699104
179481.19488260534
167002.05847074004
155872.1331894487
145913.5595580687
136977.3979961614
128938.90284081674
121693.19107443946
115150.49082087439
109231.20253042804
103862.13326780344
98975.27273978114
94509.25773028395
90411.94401811282
86641.73170324934
83166.3852654692
79960.05310366194
77000.10960998695
74265.13038617113
71734.41256286485
69388.5035485303
67209.88048560567
65183.292794561676
63295.73769645199
61536.18643270392
59895.12242538775
58363.93067724019
56934.259

8097.134539672942
8077.866386647369
8058.675042747622
8039.559972083393
8020.52063895654
8001.556507921338
7982.667043852647
7963.851710875581
7945.109974799534
7926.441300911985
7907.845155287169
7889.321006072103
7870.868321367207
7852.486571696172
7834.175226819816
7815.933759363989
7797.761645040341
7779.658359568286
7761.623382310792
7743.65619436196
7725.756281072606
7707.923131283256
7690.156236591971
7672.455092827256
7654.819201496139
7637.24806802473
7619.741203237116
7602.298123762462
7584.918352435456
7567.601418706668
7550.346857992658
7533.154215206773
7516.02304095282
7498.952893992178
7481.943343638269
7464.993964926196
7448.104344056404
7431.274076599416
7414.502765718815
7397.790026441004
7381.135482794938
7364.53876898674
7347.99952952224
7331.517418324702
7315.092101780503
7298.723254848948
7282.410564069492
7266.153725641296
7249.952447405447
7233.806447899104
7217.715454475479
7201.679207207588
7185.697457963771
7169.769964792647
7153.896502480532
7153.89650248053

1102.8553626752787
1100.753634886329
1096.7605929006013
1094.8404367680032
1096.911784795394
1098.5743769647297
1093.5890670387862
1090.0154989598848
1091.9409563803356
1091.9347585052656
1085.9058482085397
1083.3306592719339
1085.6408145449468
1083.930693465129
1079.5012598460248
1078.078413423135
1078.4966602148183
1075.0676552559796
1073.0781336060136
1071.1343308400665
1069.30132908913
1068.2826891612553
1068.223570556197
1064.993407511487
1063.7807472056263
1066.4200725428018
1067.046771266175
1060.0543093625088
1056.5385726708362
1057.8145877508396
1058.763211456619
1055.0713268257346
1052.2464764387046
1055.0341115404772
1056.2457028293263
1050.7241206989518
1045.9635474174802
1049.7147860545383
1054.5374853378519
1049.4252818246334
1044.4750291639723
1042.2981764535373
1041.4504971983097
1042.427737718444
1038.8935630111264
1034.9407857970846
1034.8257274314494
1035.8166206946717
1037.8623777977155
1031.6186271041715
1029.1403815155513
1030.7694477548466
1027.9811640851576
1028

-937.6067611634161
-937.9210524600028
-938.2831191538243
-938.4763945461975
-938.7301465924178
-939.158925234573
-939.5677019364348
-939.7934029453638
-939.9844298609278
-940.3864966955675
-940.7819758100555
-941.0252196281283
-941.3471343381618
-941.682862335371
-941.9305186408167
-942.2638106338497
-942.6077940567783
-942.8375755262573
-943.1058045655757
-943.4284498031237
-943.6481735670621
-943.8524904903729
-944.197174917467
-944.5007659304612
-944.5812650627047
-944.724169970599
-945.2663557917099
-945.701118498577
-945.7741554345582
-946.0724558816339
-946.5277010175159
-946.6776171504539
-946.890170724411
-947.2021296852115
-947.2322653265098
-947.5731373234014
-948.1531599683374
-948.2457221083221
-948.4381952272462
-948.9143843815145
-949.0921343153952
-949.2887787977857
-949.6846299864843
-949.8967844682646
-950.1117180086005
-950.4519924102071
-950.6774186255143
-950.9087626265439
-951.2214039989954
-951.4519354597235
-951.6880201301027
-951.9717474555823
-952.1944211025931

-1016.7970410089451
-1017.0105160222507
-1017.143620186727
-1017.2259154088007
-1017.2221903385805
-1017.183750465545
-1017.2391045339057
-1017.4723572723897
-1017.7781859780732
-1017.934772664994
-1017.9479203428137
-1017.9700935214275
-1018.0700331992232
-1018.2298452624888
-1018.348204570905
-1018.3974954200648
-1018.4621584383874
-1018.6198760865597
-1018.9085730927029
-1019.1309442005557
-1019.1938596431604
-1019.1933706952207
-1019.213262507375
-1019.3406327784178
-1019.5332765434838
-1019.6922802198585
-1019.8095010164359
-1019.9404565731908
-1020.1295272527858
-1020.308162570712
-1020.3981059622114
-1020.4268691377449
-1020.438754178826
-1020.4338129119527
-1020.318075429683
-1020.2740602272519
-1020.637658735734
-1021.144722272163
-1021.3553474721225
-1021.2481565540128
-1021.0655480042731
-1021.1141114418524
-1021.4739770726919
-1021.8273451649031
-1021.9160006833181
-1021.8980018119412
-1021.9837792042023
-1022.1499335680292
-1022.3132805502855
-1022.4287534589697
-1022.5240

...

6945.108650881904
6931.5058035915235
6917.959906487779
6904.470517688497
6891.037200262403
6877.65952306631
6864.3370607495235
6851.069392959752
6837.856106716164
6824.696793209902
6811.591050155219
6798.538480177302
6785.538692336158
6772.591301279076
6759.6959256477385
6746.852191099586
6734.0597294358195
6721.318173963559
6708.62716777597
6708.62716777597
6603.401506271582
6558.779392452783
6081.358966716339
5841.722940525108
5699.138866328794
5583.475133062631
5502.833628456037
5443.5303076865475
5390.129108672905
5332.132421776707
5264.958250285426
5188.291259605016
5103.117952961467
5012.485425786013
4919.818154928315
4827.915400866898
4739.987865881184
4658.453852665919
4584.122311993533
4517.798966991402
4458.276332528559
4403.76765818381
4353.4416465160375
4306.2427139189895
4260.390982434032
4215.113025730669
4170.026097687343
4125.17745959091
4080.5969040861974
4036.2480288411907
3992.9889126151634
3951.5077100914787
3911.9012732914057
3874.453427670935
3839.52981920778
3806

2074.6476477669717
2067.1775671556006
2071.672978230021
2069.100183931552
2067.7902205952823
2065.5347915005077
2063.6618369712446
2060.590984951566
2059.521510452404
2059.0282719306624
2055.7556942033
2052.9117236928914
2052.749249975269
2050.3756382418455
2052.5342090845193
2052.708004387144
2049.187855654552
2047.8959520424776
2045.227316124917
2045.5476877866272
2044.8710998502086
2045.687788519029
2042.3976078068174
2041.1520535946268
2040.9563767609848
2038.4891125153126
2041.1996052165405
2039.8318474608552
2036.8624187320856
2033.684384252971
2032.3419013355126
2029.2345482380324
2027.492352991706
2025.4771264278622
2025.04921222083
2023.833828568012
2020.101312622252
2020.5699721715728
2024.0201608542086
2026.6751419687034
2025.5560970672534
2022.401019731541
2020.2305592688733
2016.2955266136807
2017.187324027547
2018.9925682641879
2015.9690926069077
2014.5885929803917
2015.517729299584
2015.0539226883802
2014.9791800389717
2009.7316426010898
2008.4414030561124
2009.297729153

156.80690380523424
156.5829208443185
156.36099701121947
156.14115383805972
155.92340575463942
155.7077596324197
155.4942144777518
155.28276115047623
155.07338228411055
154.86605227108765
154.6607373826571
154.45739608093012
154.25597935984695
154.0564313601958
153.8586899911748
153.66268777210945
153.468352688404
153.2756091553679
153.08437907112807
152.89458284904293
152.70614045213648
152.5189724191007
152.33300081770713
152.14815011535154
151.9643479439049
151.78152574195022
151.5996192672898
151.41856895977446
151.23832019657536
151.058823412119
150.88003407077633
150.70191256732392
150.52442403035397
150.34753805458087
150.17122836790531
149.99547250008163
149.82025139560196
149.64554901908076
149.47135198888867
149.29764922332782
149.12443156077865
148.9516914869567
148.77942279237777
148.6076203470388
148.4362798395
148.26539761036088
148.09497042087537
147.92499537424413
147.75546982082034
147.58639154074422
147.4177598791136
147.24958128245027
147.08189547600983
146.9149029428

101.51980728985754
101.47801275065342
101.34050237248748
101.1950007427032
101.08510681301601
101.03606871273354
100.97179718218558
100.86432301485287
100.73948467808044
100.63455323238028
100.58570411728854
100.53694915508223
100.47339481190035
100.33306870214675
100.18040532880998
100.05403709790252
99.97624867619757
99.93190278805966
99.90214430000084
99.88963758923798
99.81692011216178
99.72533984233087
99.55789563793357
99.43813964748928
99.37044289340396
99.34372945885528
99.28465253876507
99.1503713558065
98.97934968469791
98.85768021949059
98.81314986293097
98.81044140183126
98.80837686635968
98.72243051802647
98.55721790698185
98.38422657122905
98.31673980819826
98.31090535241141
98.29638271296578
98.18246390314454
97.99607648746296
97.85103541727695
97.7994881920425
97.79706655020296
97.75872477387416
97.65851318283919
97.48977473166504
97.35524204866431
97.28209763658322
97.25162033063064
97.23159904319421
97.18289798272352
97.06540107603786
96.92453788951272
96.810010625540

70.15106896935187
70.06729247439131
70.01713345403893
70.01457778850029
69.99568959484895
69.92428709477683
69.82305016715895
69.73648019952887
69.68624493685574
69.65246748643102
69.58459110619674
69.49354821465492
69.4150800517194
69.38286566253379
69.4220333872257
69.50166077167302
69.60841242791643
69.4674306596258
69.24332891211584
69.11983959866221
69.18820421905136
69.25377782889383
69.0960228318033
68.90850138831382
68.87783835712422
68.91062546282309
68.86662684530494
68.7191441698843
68.61686970049453
68.59849341950468
68.60656427959782
68.55460233565577
68.45794576728248
68.36699769059418
68.30801456895561
68.256005533102
68.21315592358482
68.15679753109403
68.12802169774105
68.10045188244032
68.08359273184354
68.05198235312822
68.00132604314747
67.93928346852461
67.90530169520684
67.85426308702924
67.85860933062824
67.80853019837787
67.70506165081599
67.56186138766812
67.49005999257326
67.46407532955739
67.46652348457772
67.47397522096304
67.44286016475235
67.47568776712075

In [4]:
def test_run(model, seq, win_out, win_in, test_data, control_signal, path, MEAN_PRED=True, with_control=True):
    if with_control:
        YD = np.shape(test_data[0])[-1]; print('Observationa dimension:', YD)
        UD = np.shape(control_signal[0])[-1] ; print('Control signal dimension:', UD)
    
        m = RGP(wins=[win_out], with_control=True, X_dims=[YD], U_dim=UD, U_win=win_in, 
            num_inducing=100, kernels=[RBF(win_out*YD+win_in*UD, ARD=True)], 
            ctx=ctx, dtype=np.float64)
        #'./models/fully_observed/'
        m.load_params(path+model)
    
        y_pd = m.layers[0].freerun(test_data[seq].shape[0]-win_out, init_X=test_data[seq][:win_out,:],
                               U=control_signal[seq], mean_predict=False, nSamples=100)
    
        y_pd_mean = y_pd.mean(0) # the first dimention of the matrix is number of samples
        y_pd_std = y_pd.std(0)
    
    if MEAN_PRED:
        return y_pd_mean
    else:
        return y_pd_mean + y_pd_std*np.random.normal(0,0.5,size=(y_pd_mean.shape[0], y_pd_mean.shape[1]))

In [5]:
y_test_list = []
for j, bone in enumerate(y_bones):  
    y_test_list.append(y_test_dict[bone])
    
u_test_list = [np.ones((np.shape(y_test[0][0])[0],1)), np.ones((np.shape(y_test[0][1])[0],1)),
               np.ones((np.shape(y_test[0][2])[0],1)), np.ones((np.shape(y_test[0][3])[0],1)),
               np.ones((np.shape(y_test[0][0])[0],1)), np.ones((np.shape(y_test[0][1])[0],1)),
               np.ones((np.shape(y_test[0][2])[0],1)), np.ones((np.shape(y_test[0][3])[0],1))]

## collect predictions from different dimensions and concatenate into a large matrix
## This is the correct order
bones = ['root', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rfemur', 'rtibia', 'rfoot', 'rtoes', 
        'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 
        'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb',
        'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rfingers', 'rthumb']
u_bones = ['None', 'root', 'lfemur', 'ltibia', 'lfoot', 'root', 'rfemur', 'rtibia', 'rfoot', 
          'root', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck',
          'thorax', 'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lwrist',
          'thorax', 'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rwrist' ]

win_out = 20
win_in = 20
control_signal_dict = {}
num_train_seq = np.shape(y_test_list)[1]
y_pds = [[],[],[],[], [], [], [], []]
for i, model in enumerate(bones):
    print ("="*10 + model + "="*10)
    if i==0:
        u_test = u_test_list
    else:
        u_test = control_signal_dict[u_bones[i]]
    u_test_list = []
    for seq in range(num_train_seq):
        y_pd_temp = test_run(model, seq, win_out, win_in, y_test_list[i], u_test, 
                             path="./models/fully_observed/", MEAN_PRED=True, with_control=True) # output array
        u_test_list.append(y_pd_temp)
        if i==0:
            y_pds[seq] = y_pd_temp
        else:
            y_pds[seq] = np.hstack((y_pds[seq], y_pd_temp))
    control_signal_dict[model] = u_test_list
    #print(control_signal_dict)



==========root==========
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
==========lfemur==========
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
==========ltibia==========
Observationa dimension: 1
Control signal dimension: 3
Obs

Observationa dimension: 1
Control signal dimension: 3
==========lwrist==========
Observationa dimension: 1
Control signal dimension: 1
Observationa dimension: 1
Control signal dimension: 1
Observationa dimension: 1
Control signal dimension: 1
Observationa dimension: 1
Control signal dimension: 1
Observationa dimension: 1
Control signal dimension: 1
Observationa dimension: 1
Control signal dimension: 1
Observationa dimension: 1
Control signal dimension: 1
Observationa dimension: 1
Control signal dimension: 1
==========lhand==========
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
==========lfingers==========


 /home/yw440/MXRGP/mxgpy/util.py:182: RuntimeWarning:invalid value encountered in greater


Observationa dimension: 1
Control signal dimension: 2
Observationa dimension: 1
Control signal dimension: 2
Observationa dimension: 1
Control signal dimension: 2
Observationa dimension: 1
Control signal dimension: 2
Observationa dimension: 1
Control signal dimension: 2
Observationa dimension: 1
Control signal dimension: 2
Observationa dimension: 1
Control signal dimension: 2
==========lthumb==========
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
==========rclavicle==========
Observationa dimension: 2
Control signal dimension: 3
Observationa dimension: 2
Control signal dimension: 3
Observationa dimension: 2

In [12]:
mean_list = helper.reshape(y_test_mean_list)
std_list = helper.reshape(y_test_std_list)

print(np.shape(y_test_mean_list))

## write motion files
## Recover data from normalised prediction before writing motion files    
pds = [[] for _ in range(np.shape(y_test_mean_list)[-1])]

for seq, preds in enumerate(y_pds):
    pds[seq] = helper.reverse_normalise_data(preds, mean_list[seq], std_list[seq])

## lfingers and rfingers have constant values therefore, goes to 0 after normalisation. 
## mannually add original values to the final predictions.
for i in range(4):
    pds[i][:,20:21] = y_test_list[20][i] #lfingers
    pds[i][:,-4:-3] = y_test_list[27][i] #rfingers
    
for seq, pd in enumerate(pds):
    if seq < 4:
        filename = './predictions/fully_observed/walk_seq'+str(seq)
    else:
        filename = './predictions/fully_observed/run_seq'+str(seq)
    helper.write_amc(pd, filename)


(29, 8)
